# Initialize imports

In [6]:
from classes import *
from functions import *
import pandas as pd
import numpy as np

# Setting up Y_Bus, bus and line objects for all task

In [7]:
line_data = ReadCsvFile('./files/given_network/network_configuration_line_data.csv')
# bus_data = ReadCsvFile('./files/given_network/network_configuration_bus_data_slack1.csv')
bus_data = ReadCsvFile('./files/given_network/network_configuration_bus_data_slack2.csv')

Sbase = 100 # MVA
Ubase = 230 # kV
Zbase = (Ubase**2)/Sbase
max_iterations = 300
tolerance = 1e-3
Q_lim = -0.6
V_lim = -0.1 # take 1 - V_lim for max and 1 - abs(V_lim) for min

# Newton Raphson Load Flow Algorithm

In [8]:
from NRLF import NewtonRaphson

NewtonRaphson(bus_data=bus_data, 
              line_data=line_data, 
              Sbase=Sbase, 
              max_iterations=max_iterations, 
              tolerance=tolerance, 
              Q_lim=Q_lim,
              V_lim=V_lim
              )


The method converged after 3 iterations!
   Bus   Type  Voltage [pu]  Angle [deg]    P [pu]    Q [pu]
0    1     PV      1.060000    -1.313315  0.200000  1.259489
1    2  Slack      1.000000     0.000000  1.306382 -1.081814
2    3     PQ      0.987500    -3.474640 -0.450000 -0.150000
3    4     PQ      0.984242    -3.614532 -0.400000 -0.050000
4    5     PQ      0.971572    -3.941420 -0.600000 -0.100000


## Output format
X<br>Bus no.|Bus Information<br>Volts (pu)|<br>Angle (deg)|Generation<br>(MW)|Generation<br>(MVAr)|Load<br>(MW)|Load<br>(MVAr)|<br>Bus type|<br>To Bus no.|Line Flow<br>(MW)|Line Flow<br>(MVAr)|
---|---|---|---|---|---|---|---|---|---|---
|1|1.06|0|-|-|0|0|SLACK|2<br>3|||
|2|||40|-|20|10|PV|1<br>3<br>4<br>5|||
|3|||0|0|45|15|PQ|1<br>2<br>4|||
|4|||0|0|40|5|PQ|2<br>3<br>5|||
|5|||0|0|60|10|PQ|2<br>4|||


# Decoupled Load Flow

In [4]:
from DLF import DLF

DLF(bus_data=bus_data, 
    line_data=line_data, 
    Sbase=Sbase, 
    max_iterations=max_iterations, 
    tolerance=tolerance, 
    Q_lim=Q_lim,
    V_lim=V_lim
    )


The method converged after 8 iterations!

   Bus   Type  Voltage [pu]  Angle [deg]    P [pu]    Q [pu]
0    1  Slack      1.060000     0.000000  1.311197  0.908156
1    2     PV      1.000000    -2.061259  0.200000 -0.715921
2    3     PQ      0.987248    -4.636245 -0.450000 -0.150000
3    4     PQ      0.984133    -4.956608 -0.400000 -0.050000
4    5     PQ      0.971698    -5.764565 -0.600000 -0.100000


# Fast Decoupled Load Flow

In [5]:
from FDLF import FDLF

FDLF(bus_data=bus_data, 
    line_data=line_data, 
    Sbase=Sbase, 
    max_iterations=max_iterations, 
    tolerance=tolerance
    )

The method converged after 3 iterations!

   Bus   Type  Voltage [pu]  Angle [deg]    P [pu]    Q [pu]
0    1  Slack      1.060000     0.000000  1.311211  0.908152
1    2     PV      1.000000    -2.061257  0.200000 -0.715931
2    3     PQ      0.987248    -4.636463 -0.450000 -0.150000
3    4     PQ      0.984133    -4.956793 -0.400000 -0.050000
4    5     PQ      0.971697    -5.764784 -0.600000 -0.100000


# DC Power Flow

In [ ]:
from DCPF import DCPF

DCPF(bus_data=bus_data,
     line_data=line_data,
     Sbase=Sbase
     )